Econometría Aplicada. Lección 2
===============================

**Author:** Marcos Bujosa



#### Carga de algunos módulos de python



In [1]:
# Para trabajar con los datos y dibujarlos necesitamos cargar algunos módulos de python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{amsmath}')
import dataframe_image as dfi

In [1]:
from sympy.printing.preview import preview

def repr_png(tex, ImgFile):
    preamble = "\\documentclass[preview]{standalone}\n" \
        "\\usepackage{booktabs,amsmath,amsfonts}\\begin{document}"    
    preview(tex, filename=ImgFile, viewer='file', preamble=preamble, dvioptions=['-D','250'])
    #return open(ImgFile,'rb').read()

### Internat. airline passengers: monthly totals in thousands. Jan 49 – Dec 60



Leemos los datos de un fichero `csv` y generamos un `dataframe` de pandas.



In [1]:
OrigData = pd.read_csv('./database/Datasets-master/airline-passengers.csv')
OrigData['Month']=pd.to_datetime(OrigData['Month'])
OrigData=OrigData.set_index(['Month'])
print(OrigData.head())

Passengers
Month                 
1949-01-01         112
1949-02-01         118
1949-03-01         132
1949-04-01         129
1949-05-01         121

Vamos a crear un nuevo `dataframe` con los datos originales y varias transformaciones de los datos



In [1]:
TransformedData = OrigData.copy()
TransformedData['dataLog'] = np.log(OrigData['Passengers'])
TransformedData['dataLogDiff'] = TransformedData['dataLog'].diff(1)
TransformedData['dataLogDiffDiff12'] = TransformedData['dataLogDiff'].diff(12)

## Descomposición estructural de una serie temporal



Una estrategia para analizar series temporales es transformar los
datos para

1.  primero lograr que sean "***estacionarios***" y
2.  después, mediante más transformaciones, lograr una secuencia de
    "**datos *i.i.d***" (este segundo paso aún no lo hemos abordado)

(*recuerde que las expresiones "datos estacionarios" o "datos i.i.d." son un abuso del lenguaje*).



Pero existe otro enfoque que pretende descomponer la serie temporal en
los siguientes componentes *"no observables"* (o un subconjunto de
ellos):

$$\boldsymbol{y} = \boldsymbol{t} + \boldsymbol{c} + \boldsymbol{s} + \boldsymbol{e}$$

donde:

-   **La tendencia "$\boldsymbol{t}$":** recoge la lenta evolución de la
    media a *largo plazo*.

-   **El componente estacional "$\boldsymbol{s}$":** recoge las
    oscilaciones periódicas que se repiten regularmente en ciclos
    estacionales (de año en año, o de semana en semana, etc.).

-   **El componente cíclico "$\boldsymbol{c}$":** Cuando aparece
    explícitamente en el modelo, $\boldsymbol{c}$ recoge las
    oscilaciones a medio plazo. Es decir, aquellas de un plazo más largo
    que las oscilaciones estacionales, pero más corto que la tendencia
    de largo plazo. Si está ausente, dichas oscilaciones suelen aparecer
    en el componente de la tendencia, que entonces también podemos
    denominar *tendencia-ciclo*.

-   **El componente irregular "$\boldsymbol{e}$":** recoge las
    oscilaciones no captadas por el resto de componentes, ya que debe
    cumplir la siguiente identidad: $\boldsymbol{e} = \boldsymbol{y} -
      \boldsymbol{t} - \boldsymbol{c} - \boldsymbol{s}$.

Ajuste aceptable si (como poco) el componente irregular
$\boldsymbol{e}$ parece "*estacionario*".



### Tendencia determinista *lineal*



In [1]:
datosModelo1 = TransformedData[['dataLog']].copy()
nsample = len(datosModelo1)
datosModelo1['cte'] = [1]*nsample
datosModelo1['time'] = np.linspace(1, nsample, nsample)
model1 = sm.OLS(datosModelo1['dataLog'], datosModelo1[['cte', 'time']])
results1 = model1.fit()

In [1]:
import seaborn as sns
from matplotlib import rcParams
rcParams['figure.figsize'] = 15,5
plt.grid()  
ax = sns.regplot(x="time", y="dataLog", data=datosModelo1,
                 scatter_kws={"color": "black"}, line_kws={"color": "red"})
fig = ax.get_figure()
fig.savefig(image) # "image" no definido. Comentar esta línea al ejecutar el notebook

El modelo de tendencia más simple es la recta de regresión donde el
regresor es el propio índice $t$ que indica el instante del tiempo de
cada dato:

$$\ln{y_t}=\beta_1\cdot{1}+\beta_2\cdot t + e_t; \quad t=1:114$$

![img](./img/airlinepass+linearTrend.png)



In [1]:
round(results1.params['cte'],4)

In [1]:
round(results1.params['time'],4)

    $$\widehat{\ln{y_t}}=<<Cte-ajuste-tendencia-lineal()>>+<<Pte-ajuste-tendencia-lineal()>>\cdot\big(t\big), \qquad t=1:114$$

$$\widehat{\ln{y_t}}=nil+nil\cdot\big(t\big), \qquad t=1:114$$
$$\widehat{\ln{y_t}}=4.8137+0.01\cdot\big(t\big), \qquad t=1:114$$



In [1]:
# print(results.summary()) Esta es la forma habitual de ver los resultados
repr_png(results1.summary().as_latex(), image) # emplearé esta para importar los resultados como imagen png en el material de clase

![img](./img/resultsModel1.png)



In [1]:
results1.fittedvalues
results1.rsquared
print(results1.params)

# [goto error]

NameErrorTraceback (most recent call last)
<ipython-input-13-92a3b181b4ba> in <module>
----> 1 results1.fittedvalues
      2 results1.rsquared
      3 print(results1.params)

NameError: name 'results1' is not defined

In [1]:
ax = sns.lineplot(data=datosModelo1, x="time", y=results1.resid)
fig = ax.get_figure()
fig.savefig(image) # "image" no definido. Comentar esta línea al ejecutar el notebook

En este caso, el modelo 

$$\boldsymbol{y} = \boldsymbol{t} + \boldsymbol{e}$$

donde $\boldsymbol{t}$ es una tendencia lineal no es un ajuste
satisfactorio, pues el componente irregular $\boldsymbol{e}$ no parece
la realización de un proceso estacionario.

![img](./img/airlinepass+irreg.png)



In [1]:
datosModelo1['yhat'] = datosModelo1['cte']*results1.params['cte']+datosModelo1['time']*results1.params['time']
datosModelo1['ehat'] = results1.resid
datosModelo1['ehatDiff12'] = datosModelo1['ehat'].diff(12)

In [1]:
ax = sns.lineplot(data=datosModelo1, x="time", y=datosModelo1['ehatDiff12'])
fig = ax.get_figure()
fig.savefig(image) # "image" no definido. Comentar esta línea al ejecutar el notebook

Diferencia de orden 12 del componente irregular parece mostrar un componente cíclico.
![img](./img/airlinepass+irregDiff12.png)

Probemos con una tendencia cuadrática



### Tendencia determinista *cuadrática*



In [1]:
datosModelo2 = TransformedData[['dataLog']].copy()
nsample = len(datosModelo1)
datosModelo2['cte'] = [1]*nsample
datosModelo2['time'] = np.linspace(1, nsample, nsample)
datosModelo2['sq_time'] = [t**2 for t in datosModelo2['time']]
model2 = sm.OLS(datosModelo1['dataLog'], datosModelo2[['cte', 'time', 'sq_time']])
results2 = model2.fit()

In [1]:
datosModelo2['yhat'] = datosModelo2['cte']*results2.params['cte']+datosModelo2['time']*results2.params['time']+datosModelo2['sq_time']*results2.params['sq_time']
datosModelo2['ehat'] = results2.resid
datosModelo2['ehatDiff12'] = datosModelo2['ehat'].diff(12)

In [1]:
ax = sns.lineplot(data=datosModelo2, x="time", y="dataLog")
ax = sns.lineplot(data=datosModelo2, x="time", y="yhat")
fig = ax.get_figure()
fig.savefig(image) # "image" no definido. Comentar esta línea al ejecutar el notebook

$$\ln{y_t}=\beta_1\cdot{1}+\beta_2\cdot t + \beta_3\cdot t^2 + e_t; \quad t=1:114$$

![img](./img/airlinepass+quadraticTrend.png)



In [1]:
# print(results.summary()) Esta es la forma habitual de ver los resultados
repr_png(results2.summary().as_latex(), image) # emplearé esta para importar los resultados como imagen png en el material de clase

![img](./img/resultsModel2.png)



In [1]:
ax = sns.lineplot(data=datosModelo2, x="time", y=results2.resid)
fig = ax.get_figure()
fig.savefig(image) # "image" no definido. Comentar esta línea al ejecutar el notebook

![img](./img/airlinepass+irreg2.png)

En este caso, el modelo 

$$\boldsymbol{y} = \boldsymbol{t} + \boldsymbol{e}$$

donde $\boldsymbol{t}$ es una tendencia cuadrática tampoco es un
ajuste satisfactorio, pues el componente irregular $\boldsymbol{e}$
sigue sin parecer la realización de un proceso estacionario.



In [1]:
ax = sns.lineplot(data=datosModelo2, x="time", y=datosModelo2['ehatDiff12'])
fig = ax.get_figure()
fig.savefig(image) # "image" no definido. Comentar esta línea al ejecutar el notebook

La diferencia de orden 12 del componente irregular de este segundo
modelo sigue mostrando un componente cíclico.

![img](./img/airlinepass+irregDiff12.png)

Para obtener una *tendencia-ciclo* que capte este ciclo, necesitamos
procedimientos más sofisticados (TRAMO-SEATS, X13-ARIMA, STAMP, LDHR,
etc.)&#x2026; y que estiman tendencias estocásticas (en lugar de tendencias
deterministas con en los dos ejemplos vistos).

Pasemos a estimar un componente estacional



### Componente estacional determinista mediante *dummies*

